In [1]:
from osgeo import gdal, osr, gdalconst
from osgeo import ogr
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import re
import copy
import os
import cv2

In [2]:
import os
from osgeo import gdal


def swath_georeference(in_file, geo_file, target_dataset_name, out_file, out_geo_range, x_res_out, y_res_out):
    dataset = gdal.Open(in_file)
    subdatasets = dataset.GetSubDatasets()
    dataset_pos = 0
    target_pos = 0#-1
    for subdataset in subdatasets:
        subdataset_name = subdataset[0]
        if subdataset_name.endswith(target_dataset_name):
            target_pos = dataset_pos
        dataset_pos += 1
        
        if target_pos != -1:
            target_dataset = dataset.GetSubDatasets()[target_pos][0]
            vrt_file = os.path.splitext(in_file)[0] + '.vrt'
            vrt_out = gdal.Translate(vrt_file, target_dataset,
                                     format='vrt', unscale='true')
            georeferenced_data = gdal.Warp(out_file+"_"+str(target_pos+1)+".tif", vrt_out, multithread=True, outputBounds=out_geo_range,
                                           format='GTiff', geoloc=True, dstSRS="EPSG:4326",
                                           xRes=x_res_out, yRes=y_res_out, dstNodata=0.0, outputType=gdal.GDT_Float32)
            target_pos += 1
            print('The georeference of {} is complete.'.format(in_file))
            os.remove(vrt_file)
        else:
            print('{} has no the target dataset.'.format(in_file))


def main():
    file_prefix = '.hdf'
    input_directory = './hdfs/'
    output_directory = './hdf-output/'
    if not os.path.exists(output_directory):
        os.mkdir(output_directory)
    file_list = os.listdir(input_directory)
    for file_i in file_list:
        if file_i.endswith(file_prefix):
            file_name = input_directory + file_i
            out_file = output_directory + os.path.splitext(file_i)[0]# + '_geo.tiff'
            dataset_name = 'Image_Optical_Depth_Land_And_Ocean'
            geo_range = None  # [90.0, 55.0, 120.0, 25.0]
            x_res = 0.01
            y_res = 0.01
            swath_georeference(file_name, file_name, dataset_name, out_file, geo_range, x_res, y_res)


if __name__ == '__main__':
    main()



The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.2017324101641.hdf is complete.
The georeference of ./hdfs/MOD021KM.A2011181.0325.061.20173241016

In [3]:
folder_path = "./hdf-output/"
for filename in os.listdir(folder_path):
    #print(filename[-6:-4])
    if filename[-6:-4] == "_5" or filename[-6:-4] == "_8": 
        # MODDIS前7个波段的数据
        print(filename)
        hdf_name = "./hdfs/" + filename[:-6] + ".hdf"
        hdf_data = gdal.Open(hdf_name)
        solar_zenith_data = gdal.Open(hdf_data.GetSubDatasets()[14][0])
        # 太阳天顶角 它的角度范围是0-18000, 所以要除200 并化到0-pi
        zenith = np.deg2rad(
            solar_zenith_data.GetRasterBand(1).ReadAsArray() / 200)

        data = gdal.Open(folder_path + filename)
        rad_off = data.GetMetadataItem("radiance_offsets")
        rad_off = np.array(
                rad_off.replace(' ', '').split(","), dtype=np.float64)
        rad_scales = data.GetMetadataItem("radiance_scales")
        rad_scales = np.array(
                rad_scales.replace(' ', '').split(","), dtype=np.float64)
        ref_off = data.GetMetadataItem("reflectance_offsets")
        ref_off = np.array(
            ref_off.replace(' ', '').split(","), dtype=np.float64)
        ref_scale = data.GetMetadataItem("reflectance_scales")
        ref_scale = np.array(
            ref_scale.replace(' ', '').split(","), dtype=np.float64)
        band_size = data.RasterCount
        driver = gdal.GetDriverByName("GTiff")
        out_ds = driver.Create("./rad-corr/" + filename, 
                                data.RasterXSize, data.RasterYSize,
                                band_size*2, gdal.GDT_Int64)
        for i in range(1,band_size+1):
            band = data.GetRasterBand(i).ReadAsArray()
            corr_band = (band - rad_off[i-1]) * rad_scales[i-1]
            # 辐射
            out_band = out_ds.GetRasterBand(i)
            # 反射
            ref_band = out_ds.GetRasterBand(i + band_size)
            # 重采样太阳天顶角的数据
            z = cv2.resize(zenith, dsize=corr_band.shape,
                            interpolation=cv2.INTER_LINEAR)
            # 计算反射率
            ref = ref_scale[i-1] * (band - ref_scale[i-1]) / np.cos(z).T
            # 将反射率映射到扩大1000背, 因为类型是Int32
            ref = ref * 1000
            out_band.WriteArray(corr_band)
            ref_band.WriteArray(ref)
            print(ref.max(), ref.min(), ref.mean())
            ref2 = out_ds.GetRasterBand(i + band_size).ReadAsArray()
            print(ref2.max(), ref2.min(), ref2.mean())
            pass
        out_ds.SetProjection(data.GetProjection())
        out_ds.SetGeoTransform(data.GetGeoTransform())
        out_ds.FlushCache()
        out_ds = None
        pass

MOD021KM.A2011181.0325.061.2017324101641_5.tif
1352.6150066102987 -3.090690017977832e-06 290.61072989740563
1353 0 290.61047957125885
2230.32484992828 -1.1335507037525168e-06 373.30844880284707
2230 0 373.3084940191718
MOD021KM.A2011181.0325.061.2017324101641_8.tif
1519.7290279418173 -3.1262600652439585e-06 328.3977316575475
1520 0 328.3979096452941
2760.9612789306243 -1.7878082111906037e-06 301.85815196723627
2761 0 301.85805650708585
2643.8940579156847 -1.633568624626066e-06 304.9740656387131
2644 0 304.9739884043758
2385.6331045348275 -1.330045569161508e-06 198.88942906284754
2386 0 198.88934570015746
1943.8356449396815 -8.830357103446287e-07 114.77932946516616
1944 0 114.7792787784193
MOD021KM.A2012182.0200.061.2017334143411_5.tif
3691.882807551365 -3.1559880276182373e-06 127.08707693834334
3692 0 127.0870440569455
2229.8390512830792 -1.150619908984426e-06 139.94600886285636
2230 0 139.94620252824242
MOD021KM.A2012182.0200.061.2017334143411_8.tif
3758.685771563983 -3.27123350712169